# TP Clément MICHEL Webmining/NLP

In [ ]:
# Importation des librairies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
! python -m spacy download en_core_web_md

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')

In [ ]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
!pip install feedparser
!pip install newspaper3k
!python -m spacy download fr_core_news_md

# Acquisition de données par écoute d’un flux RSS

# Récupération des fichiers RSS

In [ ]:
# On récupère nos données dans un dataframe

df_echo = pd.read_json('lesechos.json')

In [ ]:
# Ce fichier contient 2687 articles différents où sont renseignés (s'ils sont présents) le nom du titre, la date, l'auteur, la catégorie, le contenu
# le lien image, le lien du fichier

df_echo.shape

(7, 2687)

In [ ]:
# (a) se premunir contre le bloquage de commande
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# (b) recuperer le fichier RSS
import feedparser as fp
url = "https://www.lemonde.fr/rss/une.xml"
data = fp.parse(url)
# (c) visualiser les elements du fichier RSS
print(data.feed.title)
print(data.feed.published)
# (d) iterer sur les entrees du flux RSS
for item in data.entries:
  print(item.title, item.published, item.link)

Le Monde.fr - Actualités et Infos en France et dans le monde
Sun, 26 Feb 2023 21:56:05 +0000
Même sans gaz russe, l’Allemagne poursuit sa croissance et sa transition énergétique Sun, 26 Feb 2023 18:00:08 +0100 https://www.lemonde.fr/economie/article/2023/02/26/meme-sans-gaz-russe-l-allemagne-poursuit-sa-croissance-et-sa-transition-energetique_6163392_3234.html
OM-PSG en direct : Paris s’impose contre Marseille et creuse l’écart qui les sépare en tête du classement Sun, 26 Feb 2023 18:45:10 +0100 https://www.lemonde.fr/sport/live/2023/02/26/om-psg-en-direct-suivez-le-classique-de-la-ligue-1_6163397_3242.html
Guerre en Ukraine : l’Alliance atlantique à l’épreuve du feu russe Fri, 24 Feb 2023 17:33:10 +0100 https://www.lemonde.fr/international/article/2023/02/24/ukraine-l-alliance-atlantique-a-l-epreuve-du-feu-russe_6163206_3210.html
Italie : au moins 59 migrants sont morts dans le naufrage de leur embarcation près des côtes Sun, 26 Feb 2023 11:30:39 +0100 https://www.lemonde.fr/internati

In [ ]:
# Pour les informations disponibles dans le flux RSS de l'exemple ci-dessous, nous avons:

# - Le titre du flux RSS: il peut être récupéré en utilisant data.feed.title
# - La date de publication du flux RSS : elle peut être récupérée en utilisant data.feed.published

In [ ]:
# Comme éléments correspondants dans la structure de données retournées par fp.parse, nous avons:

# - Le titre de l'entrée: il peut être récupéré en utilisant item.title
# - La date de publication de l'entrée: elle peut être récupérée en utilisant item.published
# - Le lien de l'entrée: il peut être récupéré en utilisant item.link

# On peut aussi rajouter si on veut le résumé de l'article avec:
# - Le résumé de l'entrée: il peut être récupéré en utilisant item.summary

In [ ]:
# On souhaite désormais lire les données du flux RSS pour les flux RSS du fichier source.json

df_source = pd.read_csv('sources.json')

In [ ]:
# Visualisation df_source
df_source

,"{""Actualit\u00e9s valeurs"": ""https://services.lesechos.fr/rss/investir-actualites-valeurs.xml""","""Investir responsable"": ""https://services.lesechos.fr/rss/investir-responsable.xml""","""March\u00e9s et Indices"": ""https://services.lesechos.fr/rss/investir-marches-indices.xml""","""Placement"": ""https://services.lesechos.fr/rss/investir-placements.xml""}"


In [ ]:
# On souhaite extraire toutes les urls de notre fichier sources json
import re

def extraction_url(s):
  pattern = re.compile(r"https://.+\.xml")
  match = pattern.search(s)
  if match:
    return match.group()
  return None

# On récupère toutes les urls issues de notre fichier source
urls = df_source.columns.map(extraction_url)

# On regarde les résultats

data = []
for i in urls:
  data = fp.parse(i)
  print(data.feed.title)
  print(data.feed.published)
  for item in data.entries:
    print(item.title, item.published, item.link)
  print(" ")

In [ ]:
# Visualisation des urls obtenues issues du fichier source
urls

Index(['https://services.lesechos.fr/rss/investir-actualites-valeurs.xml',
       'https://services.lesechos.fr/rss/investir-responsable.xml',
       'https://services.lesechos.fr/rss/investir-marches-indices.xml',
       'https://services.lesechos.fr/rss/investir-placements.xml'],
      dtype='object')

# Récupération des articles

In [ ]:
# Pour l'une des urls de notre flux RSS, on souhaite regarder comment parse la librairie newspapers dans notre cas on prend urls[3], on regarde le code HTML
import newspaper as np

article = np.Article(urls[3])
article.download()
article.parse()

print(article.html)

# C'est très agréable, on peut tout de suite visualiser les différents titres des articles, les catégories, les liens vers les articles ...

<?xml version="1.0" encoding="UTF-8"?>
<rss xmlns:atom="http://www.w3.org/2005/Atom" xmlns:media="http://search.yahoo.com/mrss/" version="2.0">
    <channel>
        <title><![CDATA[Placements]]></title>
        <description><![CDATA[Placements : assurance-vie, immobilier, impôts, retraite, famille, vie pratique, avis d‘experts]]></description>
        <link>https://investir.lesechos.fr/placements</link>
        <atom:link rel="self" href="https://services.lesechos.fr/rss/investir-placements.xml" type="application/rss+xml"/>
        <category/>
        <language>fr-FR</language>
        <copyright>Investir 2023</copyright>
        <pubDate>Sun, 26 Feb 2023 22:06:24 GMT</pubDate>
        <lastBuildDate>Sun, 26 Feb 2023 22:06:24 GMT</lastBuildDate>
        <ttl>60</ttl>
        <item>
            <title><![CDATA[La baisse des prix immobiliers est amorcée en Île-de-France]]></title>
            <pubDate>Fri, 24 Feb 2023 16:54:21 GMT</pubDate>
            <description><![CDATA[Après deux a

In [ ]:
# Peut-on identifier les zones contenant le titre de l'article?
# Oui, on peut le trouver dans le code html, par exemple ici : <title><![CDATA[Placements]]></title>

data = []
potentiels_nouveaux_articles = []
for i in urls:
  data = fp.parse(i)
  for item in data.entries:
    potentiels_nouveaux_articles.append(item.link)

# Prenons le premier article de notre source du flux RSS

article_1 = potentiels_nouveaux_articles[0]

# Sinon, on peut le récupérer en utilisant l'outil .title comme suivant:
article = np.Article(article_1)
article.download()
article.parse()
print(article.title)

« Nous allons recruter 6.000 personnes chez Equans »


In [ ]:
# le texte de l’article ? Oui, on peut le remarquer lorsque commence <description>, mais cela n'est pas intuitif
# et il est difficile de s'y repérer, pour cela on peut utiliser la librairie newspapers qui nous retourne directement
# l'information souhaité, si elle est présente comme le texte par exemple

# Toujours dans le cas de l'article qu'on a choisi

print(article.text)

Par François Monnier, Delphine Tillaux, Jacques Violet-Secondi

Publié le 24 févr. 2023 à 17:36

Au vu de vos dernières publications, peut-on considérer que les effets de la crise sur vos activités, notamment dans la construction, ont été surestimés ?

L’année 2022 peut être qualifiée de bonne. Nous avons accéléré le développement du groupe, et nos objectifs sont atteints. Nos résultats démontrent la robustesse de notre modèle économique. D’un côté, ils progressent, de l’autre, notre endettement diminue, bien sûr en isolant l’élément tout à fait particulier qu’ont représenté les emprunts contractés pour l’achat d’Equans. Dans la construction, le chiffre d’affaires a été tiré par l’international, et l’on y observe une amélioration de la performance opérationnelle et des marges. Bouygues Construction a finalement bien surmonté cette séquence de forte inflation. Pour 2023, nous faisons preuve d’une certaine prudence concernant l’évolution du chiffre d’affaires, mais, factuellement, sur la

In [ ]:
# La librairie permet de nous renseigner sur d'autres éléments quand ils sont aussi disponibles tels que les auteurs, le lien de l'image, le titre ...
print(article.title)
print(article.authors)
print(article.top_image)

« Nous allons recruter 6.000 personnes chez Equans »
[]
https://media.lesechos.com/api/v1/images/view/63f8b0b41feca72f0171fec6/1280x720/afe2957-arnaud-fevrier-002-.jpg


Mise à jour de notre base lesechos avec les nouveaux articles issus du flux RSS


In [ ]:
# On visualise notre dataframe echo, si on veut mettre à jour de nouveaux articles il faut qu'on récupère dans la mesure du possible, le texte, l'image, et on se propose de rajouter
# l'auteur s'il est présent ainsi que le titre
df_echo

,https://investir.lesechos.fr/marches/actualites/l-ue-ebauche-un-plan-pour-qualifier-le-gaz-et-le-nucleaire-d-investissements-verts-1995642.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/le-gouvernement-veut-eviter-les-penuries-de-main-d-oeuvre-dans-les-entreprises-1995527.php?xtor=RSS-24,https://investir.lesechos.fr/marches/analyses-opinions/2022-fera-la-part-belle-au-pricing-power-1995520.php?xtor=RSS-24,https://investir.lesechos.fr/marches/analyses-opinions/les-lecons-a-tirer-de-2021-1995538.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/wall-street-termine-l-annee-pres-de-ses-records-1995628.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/paris-et-new-york-au-zenith-1995547.php?xtor=RSS-24,https://investir.lesechos.fr/marches/a-ne-pas-manquer-demain/les-indices-pmi-manufacturiers-pour-commencer-l-annee-boursiere-1995597.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/malgre-un-leger-repli-en-fin-de-parcours-le-cac-40-boucle-sa-meilleure-annee-en-plus-de-20-ans-1995610.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/enquete-omicron-nouvelle-menace-pour-les-cours-du-petrole-1995604.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/l-once-d-or-poursuit-sa-progression-vendredi-1995601.php?xtor=RSS-24,...,https://investir.lesechos.fr/actions/rumeurs/generali-deux-grands-actionnaires-pourraient-chercher-un-nouveau-dg-sces-1979625.php,https://investir.lesechos.fr/actions/rumeurs/wizz-air-negocie-avec-airbus-une-commande-d-au-moins-100-appareils-sources-1979205.php,https://investir.lesechos.fr/actions/rumeurs/renault-discute-pour-mettre-fin-a-sa-coentreprise-avec-brilliance-en-chine-source-1979168.php,https://investir.lesechos.fr/actions/rumeurs/engie-a-recu-7-offres-pour-equans-vise-une-short-list-fin-septembre-sces-1979050.php,https://investir.lesechos.fr/actions/rumeurs/veolia-et-suez-vont-demander-ce-mois-ci-le-feu-vert-de-l-ue-pour-leur-fusion-source-1978773.php,https://investir.lesechos.fr/actions/rumeurs/bnp-paribas-discute-avec-agbank-d-une-jv-de-gestion-de-fortune-en-chine-sources-1978261.php,https://investir.lesechos.fr/actions/rumeurs/bnp-paribas-discute-avec-agbank-d-une-jv-de-gestion-de-fortune-sources-1978241.php,https://investir.lesechos.fr/actions/rumeurs/le-constructeur-de-voitures-electriques-rivian-vise-une-valorisation-de-70-80-mds-avec-une-ipo-source-1977592.php,https://investir.lesechos.fr/actions/rumeurs/la-chine-envisage-d-interdire-les-ipo-aux-usa-a-certaines-entreprises-technologiques-source-1977471.php,https://investir.lesechos.fr/actions/rumeurs/dws-chute-la-sec-enquete-sur-la-gestion-des-investissements-durables-source-1977342.php
title,Economie : L'UE ébauche un plan pour qualifier...,Economie : Le gouvernement veut éviter les pén...,Analyse et Stratégie : 2022 fera la part belle...,Analyse et Stratégie : Les leçons à tirer de 2...,Marchés américains : Wall Street termine l'ann...,Rétro de la semaine : Paris et New York au zénith,A ne pas manquer demain : Les indices PMI manu...,Paris Clôture : Malgré un léger repli en fin d...,Matières premières : Pétrole Brent : Enquête :...,Or : L'once d'or poursuit sa progression vendredi,...,GENERALI ASS : Generali-Deux grands actionnair...,AIRBUS : Wizz Air négocie avec Airbus une comm...,RENAULT : Renault discute pour mettre fin à sa...,"ENGIE : Engie a reçu 7 offres pour Equans, vis...",VEOLIA ENVIRONNEMENT : Veolia et Suez vont dem...,BNP PARIBAS : BNP Paribas discute avec AgBank ...,BNP PARIBAS : BNP Paribas discute avec AgBank ...,Introductions : Le constructeur de voitures él...,Introductions : La Chine envisage d'interdire ...,Marchés américains : DEUTSCHE BANK AG : DWS ch...
date,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
author,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
category,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché

In [ ]:
# Comme nous pouvons le voir nous n'avons pas de ligne avec le texte de l'article en question, on va la rajouter puis nous les récupéreront dans le code suivant

# On créer notre nouvelle ligne texte, avec comme valeurs par défaut None
a = {col: None for col in df_echo.columns}

# On ajoute notre nouvelle ligne avec indice texte
df_echo.loc['texte'] = a

# Visualisation
df_echo

,https://investir.lesechos.fr/marches/actualites/l-ue-ebauche-un-plan-pour-qualifier-le-gaz-et-le-nucleaire-d-investissements-verts-1995642.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/le-gouvernement-veut-eviter-les-penuries-de-main-d-oeuvre-dans-les-entreprises-1995527.php?xtor=RSS-24,https://investir.lesechos.fr/marches/analyses-opinions/2022-fera-la-part-belle-au-pricing-power-1995520.php?xtor=RSS-24,https://investir.lesechos.fr/marches/analyses-opinions/les-lecons-a-tirer-de-2021-1995538.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/wall-street-termine-l-annee-pres-de-ses-records-1995628.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/paris-et-new-york-au-zenith-1995547.php?xtor=RSS-24,https://investir.lesechos.fr/marches/a-ne-pas-manquer-demain/les-indices-pmi-manufacturiers-pour-commencer-l-annee-boursiere-1995597.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/malgre-un-leger-repli-en-fin-de-parcours-le-cac-40-boucle-sa-meilleure-annee-en-plus-de-20-ans-1995610.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/enquete-omicron-nouvelle-menace-pour-les-cours-du-petrole-1995604.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/l-once-d-or-poursuit-sa-progression-vendredi-1995601.php?xtor=RSS-24,...,https://investir.lesechos.fr/actions/rumeurs/generali-deux-grands-actionnaires-pourraient-chercher-un-nouveau-dg-sces-1979625.php,https://investir.lesechos.fr/actions/rumeurs/wizz-air-negocie-avec-airbus-une-commande-d-au-moins-100-appareils-sources-1979205.php,https://investir.lesechos.fr/actions/rumeurs/renault-discute-pour-mettre-fin-a-sa-coentreprise-avec-brilliance-en-chine-source-1979168.php,https://investir.lesechos.fr/actions/rumeurs/engie-a-recu-7-offres-pour-equans-vise-une-short-list-fin-septembre-sces-1979050.php,https://investir.lesechos.fr/actions/rumeurs/veolia-et-suez-vont-demander-ce-mois-ci-le-feu-vert-de-l-ue-pour-leur-fusion-source-1978773.php,https://investir.lesechos.fr/actions/rumeurs/bnp-paribas-discute-avec-agbank-d-une-jv-de-gestion-de-fortune-en-chine-sources-1978261.php,https://investir.lesechos.fr/actions/rumeurs/bnp-paribas-discute-avec-agbank-d-une-jv-de-gestion-de-fortune-sources-1978241.php,https://investir.lesechos.fr/actions/rumeurs/le-constructeur-de-voitures-electriques-rivian-vise-une-valorisation-de-70-80-mds-avec-une-ipo-source-1977592.php,https://investir.lesechos.fr/actions/rumeurs/la-chine-envisage-d-interdire-les-ipo-aux-usa-a-certaines-entreprises-technologiques-source-1977471.php,https://investir.lesechos.fr/actions/rumeurs/dws-chute-la-sec-enquete-sur-la-gestion-des-investissements-durables-source-1977342.php
title,Economie : L'UE ébauche un plan pour qualifier...,Economie : Le gouvernement veut éviter les pén...,Analyse et Stratégie : 2022 fera la part belle...,Analyse et Stratégie : Les leçons à tirer de 2...,Marchés américains : Wall Street termine l'ann...,Rétro de la semaine : Paris et New York au zénith,A ne pas manquer demain : Les indices PMI manu...,Paris Clôture : Malgré un léger repli en fin d...,Matières premières : Pétrole Brent : Enquête :...,Or : L'once d'or poursuit sa progression vendredi,...,GENERALI ASS : Generali-Deux grands actionnair...,AIRBUS : Wizz Air négocie avec Airbus une comm...,RENAULT : Renault discute pour mettre fin à sa...,"ENGIE : Engie a reçu 7 offres pour Equans, vis...",VEOLIA ENVIRONNEMENT : Veolia et Suez vont dem...,BNP PARIBAS : BNP Paribas discute avec AgBank ...,BNP PARIBAS : BNP Paribas discute avec AgBank ...,Introductions : Le constructeur de voitures él...,Introductions : La Chine envisage d'interdire ...,Marchés américains : DEUTSCHE BANK AG : DWS ch...
date,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
author,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
category,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché

In [ ]:
# On reprend notre code et on ré - actualisé notre table de données avec les nouveaux articles

import re

# On extrait les urls du fichier source
def extraction_url(s):
  pattern = re.compile(r"https://.+\.xml")
  match = pattern.search(s)
  if match:
    return match.group()
  return None

# On récupère toutes les urls issues de notre fichier source
urls = df_source.columns.map(extraction_url)

# On stock tous les articles issues de nos urls du fichier source pour pouvoir itérer ensuite / on print pour visualiser à quoi ressemble notre flux RSS

data = []
potentiels_nouveaux_articles = []
for i in urls:
  data = fp.parse(i)
  print(data.feed.title)
  print(data.feed.published)
  for item in data.entries:
    print(item.title, item.published, item.link)
    potentiels_nouveaux_articles.append(item.link)
  print(" ")


# On stock la dimension de notre df_echo avant modification car on divisera notre df, et on se focalisera sur les nouveaux articles
# après pour réduire les temps de computation élevées

dimension_avant = df_echo.shape[1]

# Dans le code suivant, on itère sur les colonnes, on regarde si l'article est présent, si non, alors on créer une nouvelle colonne avec l'article en question, initialisé a null partout
# Puis, on se sert de la librairie newspapers pour ajouter le titre, auteur, le lien de l'image de l'article, le texte de l'article à notre nouvelle colonne

for i in potentiels_nouveaux_articles:
  if all(j != i for j in df_echo.columns):
      print("L'article source suivant n'est pas présent dans notre base des articles échos:" , i)
      new_column_name = i
      df_echo = df_echo.assign(**{new_column_name: [None] * len(df_echo)})
      article = np.Article(i)
      article.download()
      article.parse()
      index_author = 'author'
      index_title = 'title'
      index_image = 'image_link'
      index_texte = 'texte'
      df_echo.at[index_author, new_column_name] = article.authors
      df_echo.at[index_title, new_column_name] = article.title
      df_echo.at[index_image, new_column_name] = article.top_image
      df_echo.at[index_texte, i] = article.text
  else:
      print("L'article source suivant est déjà présent dans notre base des articles échos:" , i)

# Si on voulait on pourrait itérer à nouveau sur notre table globale pour ajouter leur textes associés aux textes déjà présents
# avec la meme boucle mais i in df_echos, mais cela prend trop de temps
# à tourner et ce n'est pas notre objectif pour le moment.

# Le code suivant marche mais dure longtemps 30 mn mais affecte à chaque article un texte, regarde si l'article est téléchargable sinon enlève l'article de notre base

#for i in df_echo.columns:
#  article = np.Article(i)
#  try:
#    article.download()
#    article.parse()
#    index_texte = 'texte'
#    df_echo.at[index_texte, i] = article.text
#  except:
#    df_echo = df_echo.drop(columns = [i])

In [ ]:
# Visualisation de notre nouvelle base echo mise à jour
# Ancienne dimension de notre table : 8 x 2687
# Après mise à jour, nous avons enrichi de 79
df_echo.shape

(8, 2766)

In [ ]:
# On visualise notre nouvelle base

df_echo

,https://investir.lesechos.fr/marches/actualites/l-ue-ebauche-un-plan-pour-qualifier-le-gaz-et-le-nucleaire-d-investissements-verts-1995642.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/le-gouvernement-veut-eviter-les-penuries-de-main-d-oeuvre-dans-les-entreprises-1995527.php?xtor=RSS-24,https://investir.lesechos.fr/marches/analyses-opinions/2022-fera-la-part-belle-au-pricing-power-1995520.php?xtor=RSS-24,https://investir.lesechos.fr/marches/analyses-opinions/les-lecons-a-tirer-de-2021-1995538.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/wall-street-termine-l-annee-pres-de-ses-records-1995628.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/paris-et-new-york-au-zenith-1995547.php?xtor=RSS-24,https://investir.lesechos.fr/marches/a-ne-pas-manquer-demain/les-indices-pmi-manufacturiers-pour-commencer-l-annee-boursiere-1995597.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/malgre-un-leger-repli-en-fin-de-parcours-le-cac-40-boucle-sa-meilleure-annee-en-plus-de-20-ans-1995610.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/enquete-omicron-nouvelle-menace-pour-les-cours-du-petrole-1995604.php?xtor=RSS-24,https://investir.lesechos.fr/marches/actualites/l-once-d-or-poursuit-sa-progression-vendredi-1995601.php?xtor=RSS-24,...,https://investir.lesechos.fr/placements/immobilier/le-pea-pme-enfin-competitif-pour-le-crowdfunding-immobilier-1905169,https://investir.lesechos.fr/placements/immobilier/immobilier-est-ce-encore-le-bon-moment-pour-vendre/immobilier-est-ce-encore-le-bon-moment-pour-vendre-1904352,https://investir.lesechos.fr/marches-indices/bitcoin-cryptomonnaies/le-royaume-uni-explore-la-possibilite-dun-britcoin-future-livre-numerique-1904349,https://investir.lesechos.fr/placements/assurance-vie/le-senat-pour-la-possibilite-de-changer-dassurance-vie-et-de-compagnie-1902661,https://investir.lesechos.fr/placements/impots/fiscalite-ce-qui-change-pour-vos-impots-en-2023/fiscalite-ce-qui-change-pour-vos-impots-en-2023-1902146,https://investir.lesechos.fr/placements/quotidien/succession-dune-famille-recomposee-1900855,https://investir.lesechos.fr/marches-indices/bitcoin-cryptomonnaies/4-chiffres-a-retenir-a-propos-des-nft-en-2022-1900308,https://investir.lesechos.fr/marches-indices/bitcoin-cryptomonnaies/les-nouveaux-acteurs-des-cryptos-obtiennent-un-repit-mais-plus-dobligations-1900214,https://investir.lesechos.fr/placements/quotidien/chouchou-des-francais-le-livret-a-ne-connait-pas-la-crise/le-livret-a-et-le-ldds-connaissent-un-nouveau-record-a-pres-de-510-milliards-deuros-1900015,https://investir.lesechos.fr/placements/immobilier/la-duree-des-prets-immobiliers-na-jamais-ete-aussi-longue-1898756
title,Economie : L'UE ébauche un plan pour qualifier...,Economie : Le gouvernement veut éviter les pén...,Analyse et Stratégie : 2022 fera la part belle...,Analyse et Stratégie : Les leçons à tirer de 2...,Marchés américains : Wall Street termine l'ann...,Rétro de la semaine : Paris et New York au zénith,A ne pas manquer demain : Les indices PMI manu...,Paris Clôture : Malgré un léger repli en fin d...,Matières premières : Pétrole Brent : Enquête :...,Or : L'once d'or poursuit sa progression vendredi,...,Le PEA-PME enfin compétitif pour le crowdfundi...,Immobilier : est-ce encore le bon moment pour ...,Le Royaume-Uni explore la possibilité d’un « B...,Le Sénat pour la possibilité de changer d’assu...,Fiscalité : ce qui change pour vos impôts en 2023,Succession d'une famille recomposée,4 chiffres à retenir à propos des NFT en 2022,Les nouveaux acteurs des cryptos obtiennent un...,Le Livret A et le LDDS connaissent un nouveau ...,La durée des prêts immobiliers n'a jamais été ...
date,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
author,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
category,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché,Infos des Marché

Comment enregistrer nos images de nos articles?

In [ ]:
# Les images  ́etant susceptibles de ne pas rester sur le site web, il faudrait id ́ealement en faire une
# copie locale dans notre base de données pour pouvoir s’y référer ultérieurement.



# Dans un premier temps, il nous faut créer notre dossier/ou avoir accès à echos-db pour y déposer nos futures images.
# Ensuite, on va itérer sur notre dataframe, et récuperer 'l'image link' de chaque article
# Après il existe différentes librairies en python efficace, comme pillow ou request avec par exemple .requests.get afin de télécharger nos images issues des urls obtenus précédements
# Après, il nous faut donner un nom à chacune de nos images et qui soit unique pour qu'on puisse y accéder, si on en a besoin plus tard
# On enregistre dans notre dossier notre image avec son nom
# En suite on peut rajouter directement dans notre dataframe au niveau image_file, notre copie locale de l'image

In [ ]:
# On souhaite Donner une idée d’une (ou plusieurs) autre(s) source(s) que l’on pourrait-on écouter pour col-
# lecter des informations complémentaires sur ces articles ? Sans donner le code, donnez une idée
# de ce que l’on pourrait collecter sur ces sources et de la/des librairie(s) qui pourrai(en)t nous aider en cela.

# Dans un premier temps, on pourrait finir de rajouter les auteurs lorsqu'ils sont présents, les dates de publications, le contenue les catégories à nos articles dans notre base de données
# tout ceux-ci avec les mêmes librairies newspapers et feedparser

# Une première idée afin de rajouter des informations, on pourrait récolter dans la mesure du possible les commentaires sous les articles
# pour ce faire, il existe plusieurs librairies comme Beautiful Soup ou Scrapy pour extraire les commentaires directement depuis le code HTML.

# On pourrait récolter des résultats de recherche sur Google avec l'aide de la librairie Google Search API

# Ou bien encore rechercher des articles similaires...


# Partie 2:

In [ ]:
# load pipeline and related models
nlp2 = spacy.load("fr_core_news_md")

In [ ]:
# process text with pipeline
doc = nlp2("Jean Dupont est maire de Plouguemeur. Apple n’y a pas de locaux.")
# list e that were detected
for entity in doc.ents:
  print(entity.text, entity.start, entity.end, entity.label_)

Jean Dupont 0 2 PER
Plouguemeur 5 6 LOC
Apple 7 8 ORG


In [ ]:
# Dataframe reprenant nos nombres
df_update = df_echo.iloc[:,dimension_avant::]

In [ ]:
df_update

,https://investir.lesechos.fr/actu-des-valeurs/entretiens/nous-allons-recruter-6000-personnes-chez-equans-1909800,https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/chiffre-daffaires-et-marges-record-en-2022-pour-saint-gobain-un-champion-de-lisolation-dans-un-monde-qui-lutte-contre-le-gaspillage-energetique-1909755,https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/fnac-darty-sapprete-a-souffrir-de-la-flambee-des-couts-de-lenergie-cette-annee-reste-la-rumeur-kretinsky-1909728,https://investir.lesechos.fr/actu-des-valeurs/etudes/neuf-rendez-vous-biote-ch-a-ne-pas-manquer/neuf-rendez-vous-biote-ch-a-ne-pas-manquer-1909527,https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/chez-essilorluxottica-le-ralentissement-majeur-de-la-rentabilite-au-second-semestre-passe-mal-en-bourse-1909386,https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/nexity-chute-encore-en-bourse-cette-fois-cest-la-prevision-de-resultat-operationnel-qui-decoit-1909369,https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/fnac-darty-en-forte-hausse-sur-un-interet-suppose-de-lhomme-daffaires-daniel-kretinsky-pour-les-parts-de-ceconomy-1909325,https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/la-biotech-abivax-leve-130-millions-deuros-soit-90-de-sa-capitalisation-boursiere-au-prix-du-marche-1909287,https://investir.lesechos.fr/actu-des-valeurs/chat-rdv-investir/chat-avec-muriel-breiman-le-27-fevrier-1908792,https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/le-gestionnaire-de-maisons-de-retraite-korian-plonge-en-bourse-lactivite-est-au-rendez-vous-mais-pas-les-marges-1908975,...,https://investir.lesechos.fr/placements/immobilier/le-pea-pme-enfin-competitif-pour-le-crowdfunding-immobilier-1905169,https://investir.lesechos.fr/placements/immobilier/immobilier-est-ce-encore-le-bon-moment-pour-vendre/immobilier-est-ce-encore-le-bon-moment-pour-vendre-1904352,https://investir.lesechos.fr/marches-indices/bitcoin-cryptomonnaies/le-royaume-uni-explore-la-possibilite-dun-britcoin-future-livre-numerique-1904349,https://investir.lesechos.fr/placements/assurance-vie/le-senat-pour-la-possibilite-de-changer-dassurance-vie-et-de-compagnie-1902661,https://investir.lesechos.fr/placements/impots/fiscalite-ce-qui-change-pour-vos-impots-en-2023/fiscalite-ce-qui-change-pour-vos-impots-en-2023-1902146,https://investir.lesechos.fr/placements/quotidien/succession-dune-famille-recomposee-1900855,https://investir.lesechos.fr/marches-indices/bitcoin-cryptomonnaies/4-chiffres-a-retenir-a-propos-des-nft-en-2022-1900308,https://investir.lesechos.fr/marches-indices/bitcoin-cryptomonnaies/les-nouveaux-acteurs-des-cryptos-obtiennent-un-repit-mais-plus-dobligations-1900214,https://investir.lesechos.fr/placements/quotidien/chouchou-des-francais-le-livret-a-ne-connait-pas-la-crise/le-livret-a-et-le-ldds-connaissent-un-nouveau-record-a-pres-de-510-milliards-deuros-1900015,https://investir.lesechos.fr/placements/immobilier/la-duree-des-prets-immobiliers-na-jamais-ete-aussi-longue-1898756
title,« Nous allons recruter 6.000 personnes chez Eq...,Chiffre d’affaires et marges record en 2022 po...,Fnac Darty s’apprête à souffrir de la flambée ...,Neuf rendez-vous biotech à ne pas manquer,"Chez EssilorLuxottica, le ralentissement « maj...","Nexity chute encore en Bourse, cette fois c’es...",Fnac Darty en forte hausse sur un intérêt supp...,"La biotech Abivax lève 130 millions d’euros, s...","Chat avec Muriel Breiman, le 27 février",Le gestionnaire de maisons de retraite Korian ...,...,Le PEA-PME enfin compétitif pour le crowdfundi...,Immobilier : est-ce encore le bon moment pour ...,Le Royaume-Uni explore la possibilité d’un « B...,Le Sénat pour la possibilité de changer d’assu...,Fiscalité : ce qui change pour vos impôts en 2023,Succession d'une famille recomposée,4 chiffres à retenir à propos des NFT en 2022,Les nouveaux acteurs des cryptos obtiennent un...,Le Livret A et le LDDS connaissent un nouveau ...,La durée 

In [ ]:
for i in df_update.columns:
    # récupération du texte de l'article
    texte_article = df_echo.loc['texte',i]
    # application de la détection d'entités nommées
    doc = nlp(texte_article)
    for entity in doc.ents:
        print(entity.text, entity.start_char, entity.end_char, entity.label_)

In [ ]:
# A quoi correspondent les étiquettes IOB utilisées ?
# Expliquer brièvement une technique (description de la tâche, du modèle) pour la détection des entités nommées.

# IOB correspond à Inside, Outside, Beginning, cela correspond à un format de balisage pour le balisage de tokens.

# Le préfixe B- devant une balise indique que la balise correspond au début d'une entité nommée
# Le préfixe I- devant une balise indique que la balise est à l'intérieur d'une entité nommée.
# Le préfixe B- n'est utilisé que lorsqu'une balise est suivie d'une balise du même type sans qu'aucun token O ne les sépare.
# Une balise O indique que le token n'appartient à aucune entité nommée

# Une tâche de détection d'entité nommée est par exemple l'extraction de relations entre nos entités dans un texte.
# On peut se reposer sur l'utilisation d'un modèle de réseau de neurones ainsi que des modèles comme Word2Vec.s

In [ ]:
# On souhaite écrire un programme qui traite l’ensemble des textes qu'on a chargé depuis le fichier
# lesechos.json et stock pour chaque article les entités trouvées et les informations afférentes
# (type, span).
# On cherchera à optimiser le temps de traitement en utilisant la commande pipe et
# en n’activant dans la chaîne de traitement spaCy uniquement les traitements nécessaires pour la
# tâche à effectuer.  C
# Chaque document ne devra être traité qu’une et une seule fois par spaCy pour l’ensemble des questions à suivre : la suite exploite le résultat de cette analyse mais ne requiert pas de refaire l’analyse linguistique de la phrase.

In [ ]:
liste_entites = []
for i in df_update.columns:
    # récupération du texte de l'article
    texte_article = df_echo.loc['texte',i]
    # application de la détection d'entités nommées
    doc = nlp2(texte_article)
    entites = [{'text':ent.text,'type': ent.label_,'span':(ent.start, ent.end)} for ent in doc.ents]
    liste_entites.append({'article': i, 'entités': entites})

In [ ]:
# On chercher à optimiser le temps de traitement en utilisant la commande pipe et
# en n’activant dans la chaîne de traitement spaCy uniquement les traitements nécessaires pour la tâche à effectuer.

# On peut se servir de la méthode disable_pipes, on reprend le code:

liste_entites = []
# La ligne suivante désactive toutes les tâches de traitement de texte sauf la détection d'entités (symbolisé par ner : named entity recognition)
nlp2.disable_pipes(*[pipe for pipe in nlp2.pipe_names if pipe != 'ner'])

for i in df_update.columns:
    # récupération du texte de l'article
    texte_article = df_echo.loc['texte',i]
    # application de la détection d'entités nommées
    doc = nlp2(texte_article)
    entites = [{'text':ent.text,'type': ent.label_,'span':(ent.start, ent.end)} for ent in doc.ents]
    liste_entites.append({'article': i, 'entités': entites})

In [ ]:
# Pour chaque type d’entité, on souhaite déterminer les 20 plus fréquentes dans la collection de documents.
# On mémorisera ces entités dans trois listes distinctes (une par type d’entitée).

# Initialisation de nos listes pour compter chacune des occurences

liste_PER = {}
liste_LOC = {}
liste_ORG = {}
liste_MISC = {}

# On itère à travers chaque article dans la liste de nos entités, et pour chaque entité nommée,
# on vérifie son type PER,LOC,ORG,MISC, et on stock nom aisni que son nombre d'occurences dans leurs dictionnaires respectifs.
# Si notre entité est déja présente, alors on incrémente le nombre d'occurences

# On utilise get() pour récupérer le nom de notre entité à partir de la clé 'nom' du dictionnaire 'ent'.
# Si notre clé n'existe pas, on retourne la valeur de la clé 'text'

for i in liste_entites:
  for ent in i['entités']:
    if ent['type'] == 'PER':
      nom_entité = ent.get('nom', ent['text'])
      if nom_entité in liste_PER:
        liste_PER[nom_entité] += 1
      else:
        liste_PER[nom_entité] = 1
    elif ent['type'] == 'LOC':
      nom_entité = ent.get('nom', ent['text'])
      if nom_entité in liste_LOC:
        liste_LOC[nom_entité] += 1
      else:
        liste_LOC[nom_entité] = 1
    elif ent['type'] == 'ORG':
      nom_entité = ent.get('nom', ent['text'])
      if nom_entité in liste_ORG:
        liste_ORG[nom_entité] += 1
      else:
        liste_ORG[nom_entité] = 1
    elif ent['type'] == 'MISC':
      nom_entité = ent.get('nom', ent['text'])
      if nom_entité in liste_MISC:
        liste_MISC[nom_entité] += 1
      else:
        liste_MISC[nom_entité] = 1

# On trie nos résultats par nombre d'occurences décroissants et on récupère les 20 premiers pour chacun de nos dictionnaires

PER_occurences = sorted(liste_PER, key= lambda x: liste_PER[x], reverse=True)[:20]
LOC_occurences = sorted(liste_LOC, key= lambda x: liste_LOC[x], reverse=True)[:20]
ORG_occurences = sorted(liste_ORG, key= lambda x: liste_ORG[x], reverse=True)[:20]
MISC_occurences = sorted(liste_MISC, key= lambda x: liste_MISC[x], reverse=True)[:20]

In [ ]:
print("Nom de personnes les plus fréquents")
PER_occurences

Nom de personnes les plus fréquents


['Denis Lantoine\n\nPublié',
 'Daniel Kretinsky',
 'John Wiburg\n\nPublié',
 'Stifel',
 'Muriel Breiman\n\nPublié',
 'Jean Luc Champetier\n\nPublié',
 'Sophie Bellon',
 'Remy Demichelis\n\nPublié',
 'Equans',
 'Stellantis',
 'Korian',
 'Nicolas Loz de',
 'Sandra Mathorel\n\nPublié',
 'Arthur Le Denn\n\nPublié',
 'Bruno Le Maire',
 'Colas',
 'Teleperformance',
 'Bertrand Piccard',
 'Gautier',
 'Mirova']

In [ ]:
print("Nom des organisations les plus fréquentes")
ORG_occurences

Nom des organisations les plus fréquentes


['Fed',
 'Fnac Darty',
 'AMF',
 'PCE',
 'FTX',
 'Valeo',
 'Oddo BHF',
 'Investir',
 'Sénat',
 'Bouygues',
 'BCE',
 'Université du Michigan',
 'EssilorLuxottica',
 'Bourse de Paris',
 'Résultats annuels',
 'NFT',
 'Finances',
 'Assemblée nationale',
 'Imerys',
 'Nasdaq']

In [ ]:
print("Nom des lieux les plus fréquents")
LOC_occurences

Nom des lieux les plus fréquents


['France',
 'Etats-Unis',
 'Bourse',
 'Europe',
 'Saint-Gobain',
 'New York',
 'Chine',
 'Brésil',
 'Français',
 'Paris',
 'Allemagne',
 'Wall Street',
 'Nasdaq Composite',
 'Inde',
 'Equans',
 'Nexity',
 'Bourse de Paris',
 'Biocera-Vet',
 'la France',
 'Beauvoir']

In [ ]:
# Les résultats obtenues sont assez logiques, lesechos étant un journal Français, les noms de personnes les plus fréquents sont les
# journalistes, souvent auteurs de l'article en question, ou des politiciens
# Les lieux sont évidemment la France, ou l'europe et les grandes puissances
# Pour les organisations, c'est aussi logique car nos articles ont une section économie d'où les noms de banques qui reviennent souvent.

In [ ]:
# On souhaite écrire une fonction qui prend en entrée une paire d'entités et les types correspondants et qui retourne le nombre de co-occurences de ces
# deux entités au sein du même document

# On parcoure nos articles issues de liste_entites, et pour chacun de ces articles, on regarde si les entités sont présentes
# si oui on incrémente de 1

def paire_occurence(entite1,entite2,type_entite1,type_entite2,liste_entites):
  count = 0
  for i in liste_entites:
    trouve1 = False
    trouve2 = False
    for ent in i['entités']:
      if ent['type'] == type_entite1 and (ent.get('nom', ent['text']) == entite1 or ent['text'] == entite1):
        trouve1 = True
      elif ent['type'] == type_entite2 and (ent.get('nom', ent['text']) == entite2 or ent['text'] == entite2):
        trouve2 = True
    if trouve1 and trouve2:
      count += 1
  return count


In [ ]:
# On essaye de voir pour quelques exemples
print(paire_occurence('Etats-Unis','France','LOC','LOC',liste_entites))
print(paire_occurence('Etats-Unis','Denis Lantoine\n\nPublié','LOC','PER',liste_entites))
print(paire_occurence('Chine','Université du Michigan','LOC','ORG',liste_entites))

9
3
2
0


In [ ]:
co_occurences = {}
for i in PER_occurences:
  for j in LOC_occurences:
    co_occurences[(i,j)] = paire_occurence(i,j,'PER','LOC',liste_entites)
for i in PER_occurences:
  for j in ORG_occurences:
    co_occurences[(i,j)] = paire_occurence(i,j,'PER','ORG',liste_entites)
for i in LOC_occurences:
  for j in ORG_occurences:
    co_occurences[(i,j)] = paire_occurence(i,j,'LOC','ORG',liste_entites)

co_occurences_frequentes = sorted(co_occurences, key = lambda x: co_occurences[x], reverse = True)

In [ ]:
# On se propose de regarder les 10 co-occurences les plus fréquentes
co_occurences_frequentes[:10]

[('John Wiburg\n\nPublié', 'Fed'),
 ('Etats-Unis', 'Fed'),
 ('Stifel', 'Etats-Unis'),
 ('John Wiburg\n\nPublié', 'Fnac Darty'),
 ('New York', 'Fed'),
 ('John Wiburg\n\nPublié', 'Etats-Unis'),
 ('Stifel', 'Saint-Gobain'),
 ('Stifel', 'Fed'),
 ('Stifel', 'PCE'),
 ('Stifel', 'Valeo')]

In [ ]:
# On voit que John Wiburg et Fed est la co-occurence la plus fréquente, on peut vérifier
print(paire_occurence('John Wiburg\n\nPublié', 'Fed','PER','ORG',liste_entites))

9


In [ ]:
# On peut ainsi supposer que John Wiburg est un journaliste spécialisé en économie, et des Etats-Unis

In [ ]:
# On veut reprendre le code précédent en se limitant aux entités d'une même phrase
# Il faut qu'on modifie notre code initial pour que notre liste_entites contiennent nos phrases

# Il faut qu'on redownload, car on avait modifié quelques paramètres, sinon on aura des erreurs

nlp2 = spacy.load('fr_core_news_md')

In [ ]:
liste_entites = []
phrases = []

for i in df_update.columns:
    # récupération du texte de l'article
    texte_article = df_echo.loc['texte',i]
    # application de la détection d'entités nommées
    doc = nlp2(texte_article)
    for sent in doc.sents:
      entites = [{'text':ent.text,'type': ent.label_,'span':(ent.start,ent.end)} for ent in sent.ents]
      # on ajoute à notre liste_entite phrases
      liste_entites.append({'article': i,'phrase': sent.text, 'entités': entites})
      # phrases de côté si besoin
      phrases.append(sent.text)

In [ ]:
liste_entites

In [ ]:
# On reprend le code au niveau phrase

liste_phrase_PER = {}
liste_phrase_LOC = {}
liste_phrase_ORG = {}


for phrase in phrases:
  # Pour chacune des phrases on refait le même procédé
  liste_entites_phrase = []
  # Il faut doc chacune des phrases
  doc = nlp2(phrase)
  entites = [{'text':ent.text,'type': ent.label_,'span':(ent.start, ent.end)} for ent in doc.ents]
  liste_entites_phrase.append({'article': phrase, 'entités': entites})

  # Puis on itère sur notre liste
  for i in liste_entites_phrase:
    for ent in i['entités']:
      if ent['type'] == 'PER':
        nom_entité = ent.get('nom', ent['text'])
        if nom_entité in liste_phrase_PER:
          liste_phrase_PER[nom_entité] += 1
        else:
          liste_phrase_PER[nom_entité] = 1
      elif ent['type'] == 'LOC':
        nom_entité = ent.get('nom', ent['text'])
        if nom_entité in liste_phrase_LOC:
          liste_phrase_LOC[nom_entité] += 1
        else:
          liste_phrase_LOC[nom_entité] = 1
      elif ent['type'] == 'ORG':
        nom_entité = ent.get('nom', ent['text'])
        if nom_entité in liste_phrase_ORG:
          liste_phrase_ORG[nom_entité] += 1
        else:
          liste_phrase_ORG[nom_entité] = 1


In [ ]:
liste_phrase_PER

In [ ]:
liste_phrase_LOC

In [ ]:
liste_phrase_ORG

In [ ]:
# Parmi les relations correspondant aux paires d’entités dont la fréquence de co-occurrence est
# élevée, on cherche à en trouver quelques unes qui peuvent ˆetre caractérisées par un verbe.

def verbes_entre_entites(entite1,entite2,type_entite1,type_entite2,liste_entites):
  verbes = []
  for phrase in phrases:
    liste_entites_phrase = []
    # Il faut doc chacune des phrases
    doc = nlp2(phrase)
    entites = [{'text':ent.text,'type': ent.label_,'span':(ent.start, ent.end)} for ent in doc.ents]
    liste_entites_phrase.append({'article': phrase, 'entités': entites})

    # Puis on itère sur notre liste
    for i in liste_entites_phrase:
        trouve1 = False
        trouve2 = False

        verbes_entre_entites = []
        # on itère pour chaque token
        for j in doc:
          if j.text == entite1 and j.ent_type_ == type_entite1:
            trouve1 = True
          elif j.text == entite2 and j.ent_type_ == type_entite2:
            trouve2 = True
            # si notre token est un verbe et qu'on a trouvé entité 1 et 2, on le met dans notre liste
          elif j.pos_ == 'VERB' and trouve1 and trouve2:
            verbes_entre_entites.append(j.text)
        if len(verbes_entre_entites) > 0:
          verbes.extend(verbes_entre_entites)
  return verbes

In [ ]:
print(verbes_entre_entites('France','Allemagne','LOC','LOC',liste_entites))
print(verbes_entre_entites('France','Europe','LOC','LOC',liste_entites))
# On peut s'attendre à des verbes comme : entendre, trouver, conclure, passer (un accord)...

# Bilan: rien du tout ..., il faudrait essayer avec d'autres paramètres ..

[]
[]


In [1]:
# Les couples et verbes identifiés à la question précédente peuvent être utiles pour trouver de nouvelles relations et/ou grouper des relations de même nature

# Cela nous permet d'identifier des relations, en fonction du verbe en question et des types de nos entités, on peut déjà avoir une piste sur la nature de la phrase
# par exemple: si on a deux locations et comme verbes: passer (un accord), conclure(etc), on peut pratiquement s'orienter instantannément vers des Pays comme entités

# Je pense qu'on peut aussi regrouper des couples d'entités entre-eux si ils possèdent un grand taux de similitude de verbes entre-eux
# Par exemple: Roi - Reine, on peut imaginer comme verbes: épouser, mariage etc, et Homme - Femme, on peut aussi retrouver ces verbes

In [ ]:
# Pour affiner cette analyse en utilisant l'arbre de dépendance syntaxique?

#  On pourrait récuperer des informations de l'arbre de dépendance syntaxique comme les relations de dépendances entre chacune
#  des entités et les verbes dans la phrase.
# On pourait se reposer sur la position des entités et du verbe pour déterminer la nature de leur relation, COI, COD ...